### 모폴로지
- 픽셀이 있는지 없는지 모양만 확인(값을 바꾸는 것이 아님) 및 변형
- 모양을 뽑기 위해 이진화(foreground, background로 나누기 위함) 진행
- 가장 기본적인 연산이 침식, 팽창

### 침식 연산
- 객체를 침식시키는 연산
  - salt(흰색) and pepper(검정색) 잡음가 같은 임펄스(툭 튀어나옴) 잡음 제거
    - 영역이 흰색이라면 검정색으로 깎아버림
  - 마스크를 입력 영상에 적용해서 1로 채워진 영역을 온전히 올려 놓을 수 없으면 해당 픽셀을 0으로 변경됨(알고리즘)

- 가운데 픽셀의 상하좌우 중 하나라도 0이라는 값을 가지면 깎여버림

In [7]:
# 모폴로지 침식
import numpy as np, cv2

def erode(img, mask=None):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0]

    mcnt = cv2.countNonZero(mask)
    for i in range(ycenter, img.shape[0] - ycenter):
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1
            x1, x2 = j - xcenter, j + xcenter + 1
            roi = img[y1:y2, x1:x2]
            temp = cv2.bitwise_and(roi, mask)
            cnt = cv2.countNonZero(temp)
            dst[i , j] = 255 if (cnt == mcnt) else 0
    return dst

image = cv2.imread("images/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Expection("영상 파일 읽기 오류")

data = [ 0, 1, 0,
         1, 1, 1,
         0 ,1 ,0]
mask = np.array(data, np.uint8).reshape(3,3)
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]

#mask = cv2.getStructuringElement(cv2.MPORPH_CROSS, (3, 3))
#mask = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

dst1 = erode(th_img, mask)
dst2 = cv2.erode(th_img, mask)

#dst2 = cv2.erode(th_img, mask, None, None, 2)
#dst2 = cv2.morphologyEX(th_img, cv2.NORPH_ERODE, mask)

cv2.imshow("image", image)
cv2.imshow("binary image", th_img)
cv2.imshow("User erode", dst1)
cv2.imshow("OpenCV erode", dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# 모폴로지 팽창
import numpy as np, cv2

def dilate(img, mask=None):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0]

    for i in range(ycenter, img.shape[0] - ycenter):
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1
            x1, x2 = j - xcenter, j + xcenter + 1
            roi = img[y1:y2, x1:x2]
            temp = cv2.bitwise_and(roi, mask)
            cnt = cv2.countNonZero(temp)
            dst[i , j] = 0 if (cnt == 0) else 255
    return dst

image = cv2.imread("images/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Expection("영상 파일 읽기 오류")

mask = np.array([[0, 1, 0],
                 [1, 1, 1],
                 [0 ,1 ,0]]).astype('uint8')

th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]

#mask = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

dst1 = dilate(th_img, mask)
dst2 = cv2.dilate(th_img, mask)

#dst2 = cv2.morphologyEX(th_img, cv2.MORPH_DILATE, mask)

cv2.imshow("image", image)
cv2.imshow("binary image", th_img)
cv2.imshow("User erode", dst1)
cv2.imshow("OpenCV erode", dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 열림 연산, 닫힘 연산
- 열림 연산: 침식 연산 $\to$ 팽창 연산
- 닫힘 연산: 팽창 연산 $\to$ 침식 연산

In [22]:
import numpy as np, cv2
from Common.filters import erode, dilate

def opening(img, mask):
    tmp = erode(img, mask)
    dst = dilate(tmp, mask)
    return dst
def closing(img, mask):
    tmp = dilate(img, mask)
    dst = erode(tmp, mask)
    return dst

image = cv2.imread("images/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Expection("영상 파일 읽기 오류")

mask = np.array([[0, 1, 0],
                 [1, 1, 1],
                 [0 ,1 ,0]]).astype('uint8')

th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]

#mask = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

dst1 = opening(th_img, mask)
dst2 = closing(th_img, mask)ㅠ
dst3 = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, mask)
dst4 = cv2.morphologyEx(th_img, cv2.MORPH_CLOSE, mask, iterations = 1)

#dst2 = cv2.morphologyEX(th_img, cv2.MORPH_DILATE, mask)

cv2.imshow("User_opening", dst1)
cv2.imshow("OpenCV_opening", dst2)
cv2.imshow("User_closing", dst3)
cv2.imshow("OpenCV_closing", dst4)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 8. 기하학 처리

## 사상

### 기하학적 처리의 기본
- ㅁㄴㅇㄹ

### 크기 변경
- 입력 영상의 가로와 세로로 크기를 변경해서 목적 영상을 만드는 방법

### 보간
- 순방향 사상으로 영상 확대 $\to$ 홀이 많이 발생
- 영상 축소 $\to$ 오버랩 발생
- 역방향 사상을 통해 홀과 오버랩 문제 해결
- 그러나 확대할 때 목적 영상의 좌표가 입력 영상의 좌표에 정수값으로 매핑되지 않으면, 픽셀값을 가져올 수 없음
- 이떄 필요한 방법이 보간법
  - 정확하게 매칭되지 않는 픽셀 위치를 배치
- 종류
  - 최근접 이웃 보간법
  - 양선형 보간법
  - 3차 회선 보간법

### 최근접 이웃 보간법
- 가장 가깝게 이웃한 입력 영상의 화소값을 가져오는 방법

### 양선형 보간법
- 거리에 비례해서 화소값 배치